
Assume we have a bunch of pictures of human faces (M different pictures), all in the same pixel dimension (e.g., all are r×c grayscale images). We can put all the picture pixel in a matrix with M columns , where each column has all rxc pixels of one picture stacked below each other in one vector.

Recall that principal component analysis (PCA) can be applied to any matrix, and the result is a number of vectors called the principal components. The various principal components constructed a vector space for which each column in the matrix ( or equivalently each picture) can be represented as a linear combination (i.e., weighted sum) of the principal components.

 The principal components are these eigenvectors in decreasing order of the eigenvalues of the covariance matrix of the pcitures matrix.
 is 1).

The physical meaning of the principal component vectors of
, or equivalently the eigenvectors of
, is that they are the key directions that we can construct the columns of matrix
. The relative importance of the different principal component vectors can be inferred from the corresponding eigenvalues. The greater the eigenvalue, the more useful (i.e., holds more information about
) the principal component vector. Hence we can keep only the most important principal component vectors. If matrix
 is the dataset for face pictures, the first K principal component vectors are the top K most important “face pictures”. We call them the eigenface picture.

For any given face picture, we can project its mean-subtracted version onto the eigenface picture using vector dot-product. The result is how close this face picture is related to the eigenface. If the face picture is totally unrelated to the eigenface, we would expect its result is zero. For the K eigenfaces, we can find K dot-product for any given face picture. We can present the result as weights of this face picture with respect to the eigenfaces. The weight is usually presented as a vector.

Conversely, if we have a weight vector, we can add up each eigenfaces subjected to the weight and reconstruct a new face. Let’s denote the eigenfaces as matrix
, which is a L×K matrix, and the weight vector
 is a column vector. Then for any
 we can construct the picture of a face as


which
 is resulted as a column vector of length L. Because we are only using the top K principal component vectors, we should expect the resulting face picture is distorted but retained some facial characteristic.

Since the eigenface matrix is constant for the dataset, a varying weight vector
 means a varying face picture. Therefore we can expect the pictures of the same person would provide similar weight vectors, even if the pictures are not identical. As a result, we may make use of the distance between two weight vectors (such as the L2-norm) as a metric of how two pictures resemble.


In [ ]:
# download the project data https://drive.google.com/file/d/16LI3ndlVi5dob2PbL97rLsnTt6KLRQxQ/view?usp=drive_link
# upload the data to your colab folder
# Read every PGM file in the zip. PGM is a grayscale image file format.
# We extract each PGM file into a byte string through image.read() and convert it into a numpy array of bytes.
# Then we use OpenCV to decode the byte string into an array of pixels using cv2.imdecode().
# The file format will be detected automatically by OpenCV. We save each picture into a Python dictionary faces for later use.

import cv2
import zipfile
import numpy as numpy

faces = {}
with zipfile.ZipFile("Faces.zip") as facezip:
    for filename in facezip.namelist():
        if not filename.endswith(".pgm"):
            continue # not a face picture
        with facezip.open(filename) as image:
            # If we extracted files from zip, we can use cv2.imread(filename) instead
            faces[filename] = cv2.imdecode(numpy.frombuffer(image.read(), numpy.uint8), cv2.IMREAD_GRAYSCALE)

#We will use 39 groups of pictures for 39 different people to create the algorithm
# We will keep out on group for testing the algorithm
# Take classes 1-39 for eigenfaces, keep entire class 40 and
# We will keep image 10 of class 39 as out-of-sample test
facematrix = []
facelabel = []
for key,val in faces.items():
    if key == "s39/10.pgm":
        continue # this is our test
    if key.startswith("s40/"):
        continue # this is our test set
    facematrix.append(val.flatten())
    facelabel.append(key.split("/")[0])


In [ ]:
# take a look on these picture of human faces, using matplotlib:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(4,4,sharex=True,sharey=True,figsize=(8,10))
faceimages = list(faces.values())[-48:-32] # take last 16 images
for i in range(16):
    axes[i%4][i//4].imshow(faceimages[i], cmap="gray")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
#Now we can perform principal component analysis on this dataset matrix.
# Create facematrix as (n_samples,n_pixels) matrix
facematrix = numpy.array(facematrix)
# Apply PCA to extract eigenfaces
n_components = 50  # Number of eigenfaces
# Center the data
mean_face = numpy.mean(facematrix, axis=0)  # Shape: (L,), e.g., (10304,)
facesmatrix_cntr = facematrix - mean_face  # Mean-subtracted matrix, shape: (N, L)
# Compute covariance matrix C = facesmatrix_cntr^T facesmatrix_cntr (N x N)
Cov = facesmatrix_cntr @ facesmatrix_cntr.T  # Shape: (N, N), e.g., (389, 389)

#############################################################################################
# Eigen decomposition##############################
# Calculate  eigenvalues, eigenvectors
# name the avariables eigenvalues, eigenvectors
#Missing line of code############################

eigenvalues, eigenvectors = numpy.linalg.eigh(Cov)

##############################################################################333


# Sort by eigenvalues in descending order
idx = numpy.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

# Compute eigenfaces: u_i = facesmatrix_cntr v_i, normalized
eigenfaces = (facesmatrix_cntr.T @ eigenvectors)[:, :n_components].T  # Shape: (L, K), e.g., (10304, 50)
eigenfaces = eigenfaces / numpy.linalg.norm(eigenfaces, axis=0, keepdims=True)  # Normalize columns

# Generate weights by projecting  as a KxN matrix (K eigenfaces, N samples)
weights = eigenfaces@ facesmatrix_cntr.T  # Shape: (K, N), e.g., (50, 389)

faceshape = list(faces.values())[0].shape
# Show the first 16 eigenfaces
fig, axes = plt.subplots(4,4,sharex=True,sharey=True,figsize=(8,10))
for i in range(16):
    axes[i%4][i//4].imshow(eigenfaces[i].reshape(faceshape), cmap="gray")
plt.show()


#From this picture, we can see eigenfaces are blurry faces, but indeed each eigenfaces holds some facial characteristics that can be used to build a picture.
#Since our goal is to build a face recognition system, we first calculate the weight vector for each inumpyut picture:


In [ ]:
# Test on out-of-sample image of existing class
query = faces["s39/10.pgm"].reshape(1,-1)
query_weight = eigenfaces @ (query - mean_face).T
euclidean_distance = numpy.linalg.norm(weights - query_weight, axis=0)
best_match = numpy.argmin(euclidean_distance)

# Test on out-of-sample image of existing class
query = faces["s39/10.pgm"].reshape(1, -1)
query_weights = eigenfaces @ (query - mean_face).T
dot_product_cosine_similarity = numpy.zeros(weights.shape[1])

######################################################################################################################

for i in range(0, weights.shape[1]):
  #calculate cosine similarity (cosine rule) = (a.b)/(|a||b|)
  #calculate dot product of (weights[column i], query weight) divided by norm(weights[row i]) then devided by norm (query weight))
  ####  Misssing line of code #################
  # name the dot product dot_product_cosine_similarity[i]

  dot_product_cosine_similarity[i]= numpy.dot(weights[:, i], query_weights) / (numpy.linalg.norm(weights[:, i]) * numpy.linalg.norm(query_weights))

#############################################################################################################################################



best_match = numpy.argmax(dot_product_cosine_similarity)
print("Best match %s with dot product %f" % (facelabel[best_match], ((weights / numpy.linalg.norm(weights, axis=0)).T @ (query_weights / numpy.linalg.norm(query_weights))).flatten()[best_match]))
# Visualize the reuslts
fig, axes = plt.subplots(1,2,sharex=True,sharey=True,figsize=(8,6))
axes[0].imshow(query.reshape(faceshape), cmap="gray");axes[0].set_title("Query");axes[1].imshow(facematrix[best_match].reshape(faceshape), cmap="gray");axes[1].set_title("Best match")
plt.show()

In [ ]:

# Test on out-of-sample image of new class
query = faces["s40/1.pgm"].reshape(1,-1)
# copy the same code for the query test from the previous test without the query line
#### Missing cell #################################
#######################

# Test on out-of-sample image of existing class
query = faces["s37/10.pgm"].reshape(1,-1)
query_weight = eigenfaces @ (query - mean_face).T
euclidean_distance = numpy.linalg.norm(weights - query_weight, axis=0)
best_match = numpy.argmin(euclidean_distance)

# Test on out-of-sample image of existing class
query = faces["s37/10.pgm"].reshape(1, -1)
query_weights = eigenfaces @ (query - mean_face).T
dot_product_cosine_similarity = numpy.zeros(weights.shape[1])

######################################################################################################################

for i in range(0, weights.shape[1]):
  #calculate cosine similarity (cosine rule) = (a.b)/(|a||b|)
  #calculate dot product of (weights[column i], query weight) divided by norm(weights[row i]) then devided by norm (query weight))
  ####  Misssing line of code #################
  # name the dot product dot_product_cosine_similarity[i]
 dot_product_cosine_similarity[i] = numpy.dot(weights[:, i], query_weights) / (numpy.linalg.norm(weights[:, i]) * numpy.linalg.norm(query_weights))


#############################################################################################################################################



best_match = numpy.argmax(dot_product_cosine_similarity)
print("Best match %s with dot product %f" % (facelabel[best_match], ((weights / numpy.linalg.norm(weights, axis=0)).T @ (query_weights / numpy.linalg.norm(query_weights))).flatten()[best_match]))
# Visualize the reuslts
fig, axes = plt.subplots(1,2,sharex=True,sharey=True,figsize=(8,6))
axes[0].imshow(query.reshape(faceshape), cmap="gray");axes[0].set_title("Query");axes[1].imshow(facematrix[best_match].reshape(faceshape), cmap="gray");axes[1].set_title("Best match")
plt.show()

###########################






# Test on out-of-sample image of existing class
query = faces["s38/10.pgm"].reshape(1,-1)
query_weight = eigenfaces @ (query - mean_face).T
euclidean_distance = numpy.linalg.norm(weights - query_weight, axis=0)
best_match = numpy.argmin(euclidean_distance)

# Test on out-of-sample image of existing class
query = faces["s38/10.pgm"].reshape(1, -1)
query_weights = eigenfaces @ (query - mean_face).T
dot_product_cosine_similarity = numpy.zeros(weights.shape[1])

######################################################################################################################

for i in range(0, weights.shape[1]):
  #calculate cosine similarity (cosine rule) = (a.b)/(|a||b|)
  #calculate dot product of (weights[column i], query weight) divided by norm(weights[row i]) then devided by norm (query weight))
  ####  Misssing line of code #################
  # name the dot product dot_product_cosine_similarity[i]
 dot_product_cosine_similarity[i] = numpy.dot(weights[:, i], query_weights) / (numpy.linalg.norm(weights[:, i]) * numpy.linalg.norm(query_weights))


#############################################################################################################################################



best_match = numpy.argmax(dot_product_cosine_similarity)
print("Best match %s with dot product %f" % (facelabel[best_match], ((weights / numpy.linalg.norm(weights, axis=0)).T @ (query_weights / numpy.linalg.norm(query_weights))).flatten()[best_match]))
# Visualize the reuslts
fig, axes = plt.subplots(1,2,sharex=True,sharey=True,figsize=(8,6))
axes[0].imshow(query.reshape(faceshape), cmap="gray");axes[0].set_title("Query");axes[1].imshow(facematrix[best_match].reshape(faceshape), cmap="gray");axes[1].set_title("Best match")
plt.show()




################
# Test on out-of-sample image of existing class
query = faces["s39/10.pgm"].reshape(1,-1)
query_weight = eigenfaces @ (query - mean_face).T
euclidean_distance = numpy.linalg.norm(weights - query_weight, axis=0)
best_match = numpy.argmin(euclidean_distance)

# Test on out-of-sample image of existing class
query = faces["s39/10.pgm"].reshape(1, -1)
query_weights = eigenfaces @ (query - mean_face).T
dot_product_cosine_similarity = numpy.zeros(weights.shape[1])

######################################################################################################################

for i in range(0, weights.shape[1]):
  #calculate cosine similarity (cosine rule) = (a.b)/(|a||b|)
  #calculate dot product of (weights[column i], query weight) divided by norm(weights[row i]) then devided by norm (query weight))
  ####  Misssing line of code #################
  # name the dot product dot_product_cosine_similarity[i]
 dot_product_cosine_similarity[i] = numpy.dot(weights[:, i], query_weights) / (numpy.linalg.norm(weights[:, i]) * numpy.linalg.norm(query_weights))


#############################################################################################################################################



best_match = numpy.argmax(dot_product_cosine_similarity)
print("Best match %s with dot product %f" % (facelabel[best_match], ((weights / numpy.linalg.norm(weights, axis=0)).T @ (query_weights / numpy.linalg.norm(query_weights))).flatten()[best_match]))
# Visualize the reuslts
fig, axes = plt.subplots(1,2,sharex=True,sharey=True,figsize=(8,6))
axes[0].imshow(query.reshape(faceshape), cmap="gray");axes[0].set_title("Query");axes[1].imshow(facematrix[best_match].reshape(faceshape), cmap="gray");axes[1].set_title("Best match")
plt.show()




# Test on out-of-sample image of existing class
query = faces["s40/10.pgm"].reshape(1,-1)
query_weight = eigenfaces @ (query - mean_face).T
euclidean_distance = numpy.linalg.norm(weights - query_weight, axis=0)
best_match = numpy.argmin(euclidean_distance)

# Test on out-of-sample image of existing class
query = faces["s40/10.pgm"].reshape(1, -1)
query_weights = eigenfaces @ (query - mean_face).T
dot_product_cosine_similarity = numpy.zeros(weights.shape[1])

######################################################################################################################

for i in range(0, weights.shape[1]):
  #calculate cosine similarity (cosine rule) = (a.b)/(|a||b|)
  #calculate dot product of (weights[column i], query weight) divided by norm(weights[row i]) then devided by norm (query weight))
  ####  Misssing line of code #################
  # name the dot product dot_product_cosine_similarity[i]
 dot_product_cosine_similarity[i] = numpy.dot(weights[:, i], query_weights) / (numpy.linalg.norm(weights[:, i]) * numpy.linalg.norm(query_weights))


#############################################################################################################################################



best_match = numpy.argmax(dot_product_cosine_similarity)
print("Best match %s with dot product %f" % (facelabel[best_match], ((weights / numpy.linalg.norm(weights, axis=0)).T @ (query_weights / numpy.linalg.norm(query_weights))).flatten()[best_match]))
# Visualize the reuslts
fig, axes = plt.subplots(1,2,sharex=True,sharey=True,figsize=(8,6))
axes[0].imshow(query.reshape(faceshape), cmap="gray");axes[0].set_title("Query");axes[1].imshow(facematrix[best_match].reshape(faceshape), cmap="gray");axes[1].set_title("Best match")
plt.show()